<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danh sách nhóm
- Đoàn Xuân Minh - MSSV: 18521093
- Lê Trần Phúc Nguyên - MSSV: 18521157
- Phan Quang Tấn - MSSV: 18521377


# Mô tả bài toán : Dự đoán chủ đề của một bài báo tiếng Anh
- Input: Một đoạn văn bản của một bài báo tiếng Anh (article)
- Output: Chủ đề của đoạn văn bản 

Code tham khảo: https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db

Code tham khảo: https://pythonspot.com/nltk-stop-words/

# Prepare dataset

Thực hiện cào dữ liệu (crawl data) từ các trang báo tiếng Anh nổi tiếng như : The Sun https://www.thesun.co.uk/ ,Dailymail https://www.dailymail.co.uk/home/index.html, Telepraph http://telegraph.co.uk/

Code Crawl : https://github.com/tankien76/CS114.K21/blob/master/CrawlerTest.ipynb

Tập dataset sau khi crawl gồm có hơn 2600 bài báo với 8 chủ đề khác nhau như : sport, tvshowbiz, femail,.... Mỗi chủ đề gồm có hơn 300 bài báo.

Xác thực và tạo ứng dụng PyDrive.

In [1]:
!pip install PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':"1o2zIVZBmqSJa8huDq_Nn8waeTksbH4KI"})
downloaded.GetContentFile('text_data.csv')

Import các thư viện:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text

In [5]:
import pandas as pd, numpy as np, re

Đọc dữ liệu bằng thư viện pandas từ file text_data.csv

In [6]:
dataset = pd.read_csv("text_data.csv", encoding='ISO 8859-1')
dataset.head()

,article,category
0,PHOTOS of McDonalds restaurants and merchandis...,lifestyle
1,"ARIESMarch 21 to April 20SATURN, the most powe...",lifestyle
2,BRITS are being tempted to fly abroad once loc...,lifestyle
3,DONNA Mitchell tried every fad diet in the boo...,lifestyle
4,FASHION fans can buy jeans and a sleeveless to...,lifestyle


Đưa dữ liệu article và category vào list tương ứng

In [7]:
features = dataset['article']
labels = dataset['category']

Thực hiện chuyển đổi các nhãn dán thành số để dễ dàng thực hiện trong khi training và test model

In [8]:
from sklearn import preprocessing
number = preprocessing.LabelEncoder()
labels= number.fit_transform(labels)

# Data Preprocessing
Tiến hành xử lý dữ liệu văn bản (article) :

- Loại bỏ các ký tự đặc biệt(dấu câu, số)
- Loại bỏ các stopword trong tiếng Anh

download punkt trong thư viện nltk để tách từ khi xử lý dữ liệu

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

download stopwords trong thư viện nltk để xử lý các từ không cần thiết trong tiếng anh

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Import thư viện nltk để sử lý ngôn ngữ tự nhiên

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Loại bỏ các ký tự đặc biệt như: dấu câu, số,...

In [12]:
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Loại bỏ các stopword (các từ không ảnh hưởng đến nội dung của văn bản) trong tiếng Anh

In [13]:
stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

# Feature Engineering

Vector hóa dữ liệu văn bản bằng TF-IDF:
- TF: tần suất xuất hiện của 1 từ trong 1 văn bản
- IDF: tần suất nghịch của 1 từ trong 1 tập văn bản
- .fit: Máy học từ vựng và tính toán idf( tần suất nghịch của 1 từ trong tập các văn bản) của train set.

Tham khảo: https://towardsdatascience.com/how-sklearns-tf-idf-is-different-from-the-standard-tf-idf-275fa582e73d

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer()
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Đưa dữ liệu văn bản về matrix để training model 

In [15]:
tf_idf_x = tfIdfVec.transform(features)

Tiến hành chia dữ liệu train và test: theo tỉ lệ 80% training và 20% testing vì khi thực hiện theo tỉ lệ 70/30 thì độ chính xác của các thuật toán giảm

In [16]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .2, random_state = 0)

# Chosing Model

Thêm thư viện sklearn gồm :
- Các model sử dụng : LinearSVC, Multinomial Naive Bayes, Random Forest

In [17]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

Model Linear support classifier

In [18]:
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print("Train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

Train accuracy:  0.9982616253802694
Test accuracy:  0.8715277777777778


Model Multinomial Naive Bayes

In [19]:
mnb = MultinomialNB()
mnb.fit(features_train, labels_train)
print("train accuracy: ",mnb.score(features_train, labels_train))
print("Test accuracy: ",mnb.score(features_test, labels_test))

train accuracy:  0.9096045197740112
Test accuracy:  0.7847222222222222


Model Random Forest Classifier

In [20]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print("train accuracy: ",rfc.score(features_train, labels_train))
print("Test accuracy: ",rfc.score(features_test, labels_test))

train accuracy:  0.9956540634506736
Test accuracy:  0.640625


=> Qua accuracy của các model, ta thấy model Linear support classifier có độ chính xác cao nhất với 89,2% và model Random Forest bị overfitting vì số lượng dataset ít.

# Đánh giá Model

Để đánh giá performance của các model thì cách tính bằng accuracy không phản ánh chính xác dữ liệu được phân loại như thê nào vì vậy chúng ta cần 1 cách tính độ chính xác khác đó là **F1 Score**

F1-score: trung bình điều hòa giữa precision và recall

F1-score = 2*(precision * recall)/(precision+recall)

**Precision** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm được phân loại là positive

Precision = TP/(TP+FP)
Với : TP(True positive), FP(False positive)

**Recall** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm thực sự là positive

Recall=TP/(TP+FN) Với TP(True positive), FN(False negative)

Code tham khảo : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

Model Linear support classifier

In [22]:
y1_pred=lsvc.predict(features_test) #Lưu kết quả dự đoán của model LinearSVC vào biến y1_pred
f1 = f1_score(labels_test, y1_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.8714691306459263


Model Multinomial Naive Bayes

In [23]:
y2_pred=mnb.predict(features_test) #Lưu kết quả dự đoán của model MultinomialNB vào biến y2_pred
f1 = f1_score(labels_test, y2_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7863145244924286


Model Random Forest Classifier

In [24]:
y3_pred=rfc.predict(features_test) #Lưu kết quả dự đoán của model RandomForest vào biến y3_pred
f1 = f1_score(labels_test, y3_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.6507895522637865


Dùng Confusion Matrix để trực quan hóa về F1 score của từng class

Code tham khảo: https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

In [25]:
confusion = confusion_matrix(labels_test, y1_pred)
print('Confusion Matrix')
print(confusion)

Confusion Matrix
[[56  5  6  7  0  0  4  8]
 [ 2 68  1  0  0  0  1  4]
 [ 4  0 47  3  1  2  1  3]
 [ 4  1  5 58  1  1  1  0]
 [ 0  0  0  0 54  0  0  0]
 [ 0  0  1  1  0 76  0  0]
 [ 3  1  1  0  0  0 76  0]
 [ 0  1  0  0  1  0  0 67]]


In [26]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(labels_test, y1_pred, target_names=['femail', 'films', 'health','lifestyle','sport','technology','travel','tvshowbiz']))


Classification Report

              precision    recall  f1-score   support

      femail       0.81      0.65      0.72        86
       films       0.89      0.89      0.89        76
      health       0.77      0.77      0.77        61
   lifestyle       0.84      0.82      0.83        71
       sport       0.95      1.00      0.97        54
  technology       0.96      0.97      0.97        78
      travel       0.92      0.94      0.93        81
   tvshowbiz       0.82      0.97      0.89        69

    accuracy                           0.87       576
   macro avg       0.87      0.88      0.87       576
weighted avg       0.87      0.87      0.87       576



In [27]:
confusion = confusion_matrix(labels_test, y2_pred)
print('Confusion Matrix')
print(confusion)

Confusion Matrix
[[32  2  0 42  0  0  1  9]
 [ 4 63  1  1  0  1  1  5]
 [ 1  0 26 29  0  1  2  2]
 [ 1  0  1 68  0  1  0  0]
 [ 0  0  0  1 53  0  0  0]
 [ 0  0  2  4  0 72  0  0]
 [ 0  1  0  8  0  0 72  0]
 [ 0  1  0  1  0  0  1 66]]


In [28]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(labels_test, y2_pred, target_names=['femail', 'films', 'health','lifestyle','sport','technology','travel','tvshowbiz']))


Classification Report

              precision    recall  f1-score   support

      femail       0.84      0.37      0.52        86
       films       0.94      0.83      0.88        76
      health       0.87      0.43      0.57        61
   lifestyle       0.44      0.96      0.60        71
       sport       1.00      0.98      0.99        54
  technology       0.96      0.92      0.94        78
      travel       0.94      0.89      0.91        81
   tvshowbiz       0.80      0.96      0.87        69

    accuracy                           0.78       576
   macro avg       0.85      0.79      0.79       576
weighted avg       0.85      0.78      0.78       576



In [29]:
confusion = confusion_matrix(labels_test, y3_pred)
print('Confusion Matrix')
print(confusion)

Confusion Matrix
[[36 13  9 14  2  2  2  8]
 [13 57  2  0  1  1  2  0]
 [14  2 28  4  1  8  2  2]
 [11  5  7 37  1  4  4  2]
 [ 1  2  2  0 49  0  0  0]
 [ 3  7  8  1  0 59  0  0]
 [ 7 12  3  5  0  3 50  1]
 [ 3  2  5  3  0  3  0 53]]


In [30]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(labels_test, y3_pred, target_names=['femail', 'films', 'health','lifestyle','sport','technology','travel','tvshowbiz']))


Classification Report

              precision    recall  f1-score   support

      femail       0.41      0.42      0.41        86
       films       0.57      0.75      0.65        76
      health       0.44      0.46      0.45        61
   lifestyle       0.58      0.52      0.55        71
       sport       0.91      0.91      0.91        54
  technology       0.74      0.76      0.75        78
      travel       0.83      0.62      0.71        81
   tvshowbiz       0.80      0.77      0.79        69

    accuracy                           0.64       576
   macro avg       0.66      0.65      0.65       576
weighted avg       0.65      0.64      0.64       576



# Chương trình test

Nhập đoạn văn bản tiếng Anh muốn dự đoán

In [43]:
print("Nhap: ")
ch=(str(input()))

Nhap: 
HOLIDAYMAKERS returning from France will have to quarantine, the government has announced.  It means 500,000 tourists have until 4am on Saturday to return to the UK to avoid two-week quarantine, the PM said.  France was taken off the "safe" list last night following crunch talks.  The PM said yesterday while on a trip to Northern Ireland: “We have got to be absolutely ruthless about this, even with our closest and dearest friends and partners. I think everybody understands that.  “We can’t be remotely complacent about our own situation. Everybody understands that in a pandemic you don’t allow our population to be reinfected or the disease to come back in.”  Malta and The Netherlands were also added to the list.  Budget airline Easyjet has said it will continue to fly to the countries that have recently been added to the UK's quarantine list.  Meanwhile deaths hit 41,347 after 18 more coronavirus fatalities.  Follow our coronavirus live blog for all the latest news and updates...

Xử lý dữ liệu được đưa vào

In [44]:
ch = re.sub(r'[^a-zA-Z]',' ',ch)
ch_tokens = word_tokenize(ch)
ch_sentence = [w for w in ch_tokens if not w in stop_words]
ch_val = tfIdfVec.transform([ch]).toarray()

Dự đoán chủ đề của văn bản đã nhập

In [45]:
ch_pred=lsvc.predict(ch_val)

In [46]:
if (ch_pred[0]==0):
  print('femail')
if (ch_pred[0]==1):
  print('films')
if (ch_pred[0]==2):
  print('health')
if (ch_pred[0]==3):
  print('lifestyle')
if (ch_pred[0]==4):
  print('sport')
if (ch_pred[0]==5):
  print('technology')
if (ch_pred[0]==6):
  print('travel')
if (ch_pred[0]==7):
  print('tvshowbiz')

health
